In [1]:
!unzip /content/drive/MyDrive/ApplePie-20230713T122206Z-001.zip

Archive:  /content/drive/MyDrive/ApplePie-20230713T122206Z-001.zip
   creating: ApplePie-20230713T122206Z-001/
   creating: ApplePie-20230713T122206Z-001/ApplePie/
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02577.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02578.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02579.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02581.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02582.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02583.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02584.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02585.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02587.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02588.jpg  
  inflating: ApplePie-20230713T122206Z-001/ApplePie/A020511XX_02593.jpg  
  inflating: ApplePie-

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
image_size = (100, 100)
batch_size = 12
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/ApplePie-20230713T122206Z-001",
    validation_split=0.2,
    subset="both",
    seed=1337,
    batch_size=batch_size,
    image_size= image_size,
    shuffle=True, color_mode='grayscale', interpolation='gaussian',
)

Found 497 files belonging to 6 classes.
Using 398 files for training.
Using 99 files for validation.


In [22]:
from sklearn.utils import class_weight
import numpy as np
train = train_ds.as_numpy_iterator()
x_train, y_train = next(train)
class_weights = class_weight.compute_class_weight(class_weight ='balanced',
                                                 classes = np.unique(y_train),
                                                 y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

{0: 2.0, 1: 1.0, 2: 0.5, 3: 2.0, 4: 0.6666666666666666, 5: 2.0}

In [15]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.15),
    ]
)

In [16]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

In [8]:
val_ds = val_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

In [17]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [18]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.8)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (1,), num_classes= 6)

In [23]:
STEPS_PER_EPOCH = len(y_train)/batch_size
epochs = 30
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)


model.compile(
    optimizer=keras.optimizers.Adam(lr_schedule),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,class_weight=class_weights
)

Epoch 1/30
34/34 [==============================] - 87s 2s/step - loss: 2.4479 - accuracy: 0.2663 - val_loss: 2.7020 - val_accuracy: 0.1515
Epoch 2/30
34/34 [==============================] - 81s 2s/step - loss: 2.2517 - accuracy: 0.2940 - val_loss: 2.4091 - val_accuracy: 0.1515
Epoch 3/30
34/34 [==============================] - 81s 2s/step - loss: 2.2793 - accuracy: 0.2563 - val_loss: 2.3264 - val_accuracy: 0.1515
Epoch 4/30
34/34 [==============================] - 86s 3s/step - loss: 2.1593 - accuracy: 0.2789 - val_loss: 2.4183 - val_accuracy: 0.1515
Epoch 5/30
34/34 [==============================] - 86s 3s/step - loss: 2.1711 - accuracy: 0.2965 - val_loss: 3.0410 - val_accuracy: 0.1515
Epoch 6/30
34/34 [==============================] - 82s 2s/step - loss: 2.1990 - accuracy: 0.2839 - val_loss: 2.1314 - val_accuracy: 0.1515
Epoch 7/30
34/34 [==============================] - 80s 2s/step - loss: 2.0289 - accuracy: 0.3216 - val_loss: 2.4492 - val_accuracy: 0.1515
Epoch 8/30
34/34 [==

In [35]:
img = keras.utils.load_img(
    "/content/ApplePie-20230713T122206Z-001/Bread/A020118XX_10931.jpg", target_size=image_size,color_mode='grayscale'
)
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array).tolist()
classes = ["ApplePie","BagelSandwich", "Bibimbop", "Bread","FriedRice","Pork"]
res = dict(zip(classes,predictions[0]))
res

1/1 [==============================] - 0s 106ms/step


{'ApplePie': 0.0263312216848135,
 'BagelSandwich': 0.01114265713840723,
 'Bibimbop': 0.09470003098249435,
 'Bread': 0.8319147229194641,
 'FriedRice': 0.00794193334877491,
 'Pork': 0.027969472110271454}